<a href="https://colab.research.google.com/github/kstyle2198/my_parsing/blob/main/Parser_to_Chromadb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 올라마 설치 및 모델 다운로드

In [ ]:
!pip install colab-xterm
%load_ext colabxterm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 4.1 MB/s eta 0:00:00


In [ ]:
%xterm
#  curl https://ollama.ai/install.sh | sh
#  ollama serve &
#  ollama pull bge-m3:latest

Launching Xterm...

<IPython.core.display.Javascript object>

In [ ]:
!pip install pdfplumber==0.11.2 PyPDF2==3.0.1
!pip install langchain==0.3.3 langchain-core==0.3.10 langchain-chroma==0.1.4 langchain-community==0.3.2 langchain-ollama==0.2.0
!pip install paddleocr==2.8.1 paddlepaddle==2.6.1
!pip install pycryptodome==3.15.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 102.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of langchain-text-splitters to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 36.0 MB/s eta

In [ ]:
import pdfplumber
from typing import Iterator
from langchain_core.documents import Document
from paddleocr import PaddleOCR
from pprint import pprint
import re
import os
import time
import pickle
from tqdm.auto import tqdm
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

In [ ]:
def create_folder_if_not_exists(folder_path:str):  # 이미지 저장 폴더 생성
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"폴더가 생성되었습니다: {folder_path}")
    else:
        print(f"폴더가 이미 존재합니다: {folder_path}")

def save_pdf_to_img(path:str, file_name:str, page_num:int):  # pdf를 png 이미지 파일로 저장
    with pdfplumber.open(path) as pdf:
        page = pdf.pages[page_num]
        im = page.to_image(resolution=150)
        # im.draw_rects(first_page.extract_words())  # 글자에 Red Box 그리기
        save_path = f"{os.getcwd()}/drive/MyDrive/images/{file_name}/{file_name}_{page_num}.png"
        im.save(save_path, format="PNG", )
    return save_path

table_settings={      # extract_tables method variable (깃헙 디폴트 세팅 참조)
    "vertical_strategy": "lines",
    "horizontal_strategy": "lines",
    "explicit_vertical_lines": [],
    "explicit_horizontal_lines": [],
    "snap_tolerance": 3,
    "snap_x_tolerance": 3,
    "snap_y_tolerance": 3,
    "join_tolerance": 3,
    "join_x_tolerance": 3,
    "join_y_tolerance": 3,
    "edge_min_length": 3,
    "min_words_vertical": 3,
    "min_words_horizontal": 1,
    "intersection_tolerance": 3,
    "intersection_x_tolerance": 3,
    "intersection_y_tolerance": 3,
    "text_tolerance": 3,
    "text_x_tolerance": 3,
    "text_y_tolerance": 3,
    # "text_*": …,
    }

def convert_header_to_separator(header: str) -> str:   # 테이블 첫줄 파싱후, 두번째 줄에 Header Line 추가 함수(마크다운 형식을 위한)
    # Use a regex to replace each header content with the appropriate number of hyphens
    separator = re.sub(r'[^|]+', lambda m: '-' * max(1, len(m.group(0))), header) # max 부분 관련 구분자는 최소 1개는 들어가야 마크다운 적용
    separator = separator.replace("||", "|-|", 1)  # 수평구분자가 최소 한개는 있어야 마크다운 적용(그냥 비어있으면 안됨)
    return separator

def table_parser(pdf_path:str, page_num:int, crop:bool=False) -> list:   # 테이블 파싱(마크다운 형식), A4상단 표준 크롭핑 적용 선택 가능(디폴트 false)
    full_table = []
    with pdfplumber.open(pdf_path) as pdf:
        # Find the examined page
        table_page = pdf.pages[page_num]
        if crop:
            bounding_box = (3, 70, 590, 770)   #default : (0, 0, 595, 841)
            table_page = table_page.crop(bounding_box, relative=False, strict=True)
        else: pass
        tables = table_page.extract_tables(table_settings = table_settings)
        # if tables:
        for table in tables:
            table_string = ''
            # Iterate through each row of the table
            for row_num in range(len(table)):
                row = table[row_num]
                # Remove the line breaker from the wrapped texts
                cleaned_row = [item.replace('\n', ' ') if item is not None and '\n' in item else 'None' if item is None else item for item in row]
                # Convert the table into a string
                table_string+=('|'+'|'.join(cleaned_row)+'|'+'\n')
                if row_num ==0:  # 첫줄 작업이면, Header Line 추가
                    header_line = convert_header_to_separator(table_string[:-1])
                    table_string+= header_line+'\n'
            # Removing the last line break
            table_string = table_string[:-1]
            full_table.append(table_string)
        return full_table

def extract_level_name(path:str) -> list:  # 폴더 구조(lv1, lv2, lv3를 metadata로 추출하는 함수)
    temp = path.split("/")  # path 예시 : ['.\\2024\\Manual\\Guidance for Autonomous Ships_2023.pdf','.\\2024\\POS\\FWG.pdf']
    lv1 = temp[4]
    if temp[5]:
        if temp[5] != temp[-1]:
            lv2 = temp[5]
            lv3 = temp[-1].replace(".pdf", "")
        else:
            lv2 = None
            lv3 = temp[-1].replace(".pdf", "")
    result = [lv1, lv2, lv3]
    return result

total_results =[]
def main_filepath_extractor(path:str) -> list:   # 폴더 트리를 리커시브하게 읽어서 전체 PDF 파일의 full 경로를 리스트에 수집
    global total_results
    all_items = os.listdir(path)
    files = [f for f in all_items if os.path.isfile(os.path.join(path, f))]
    results = [os.path.join(path, file) for file in files]
    results = [result.replace("\\", "/") for result in results]
    total_results.extend(results)
    dirs = [f for f in all_items if os.path.isdir(os.path.join(path, f))]
    if dirs:
        dirs = [path+"/" + lv2_dir for lv2_dir in dirs]
        for dir in dirs:
            main_filepath_extractor(dir)
    return total_results

def main_parser(path:str, crop:bool=False, lang:str="en") -> Iterator[Document]:  # 메인 Parsing 함수, text-extraction은 pypdf2 적용
    '''
    - pdfplumber: table, image 추출
    - pypdf2: text 추출
    - paddleocr : 이미지 pdf 줄파싱
    - lang 후보: ['ch', 'en', 'korean', 'japan', 'chinese_cht', 'ta', 'te', 'ka', 'latin', 'arabic', 'cyrillic', 'devanagari']
    '''
    full_result = []
    file_name = path.split("/")[-1].split(".")[0].strip()
    img_save_folder = os.path.join(os.getcwd(), f"{os.getcwd()}/drive/MyDrive/images/{file_name}")  # images 폴더 생성후 그 안에 file_name폴더 생성
    create_folder_if_not_exists(img_save_folder)  # 이미지 저장할 폴더 생성
    ocr = PaddleOCR(use_angle_cls=True, lang=lang)

    with pdfplumber.open(path) as pdf:
        page_number = 0  # for metadata
        for _ in tqdm(pdf.pages):
            level_names = extract_level_name(path)  # for metadata
            img_path = save_pdf_to_img(path, file_name, page_number) # for saving pdf page as png img file
            reader = PdfReader(path)
            page = reader.pages[page_number]
            text_result = page.extract_text().replace("\n", " ").replace("- ", "").replace("  ", " ")

            if len(text_result) == 0:  # 텍스트 추출 결과가 없으면, OCR 실시
                print("이미지 OCR")
                ocr_result = ocr.ocr(img_path)
                for idx in range(len(ocr_result)):
                    res = ocr_result[idx]
                    temp_result = []
                    try:
                        for line in res:
                            temp_result.append(line[1][0])
                    except: temp_result.append("Error has been occured")
                text_result = " ".join(temp_result)

            table_result = table_parser(path, page_number, crop)  # for page_content

            if table_result:
                total_page_result = ""
                for table in table_result:
                    total_page_result = text_result + "\n\n" + table   # table_result가 있으면, text_result 끝에 엔터후 이어붙이기
                    result = Document(
                        page_content=total_page_result,
                        metadata={"Page": page_number, "First Division":level_names[0], "Second Division": level_names[1], "File Name": level_names[2], "File Path": path},
                        )
            else:
                result = Document(
                    page_content = text_result,
                    metadata={"Page": page_number, "First Division":level_names[0], "Second Division": level_names[1], "File Name": level_names[2], "File Path": path},
                    )



            with open(f'/content/drive/MyDrive/Parsed_text/parsed_text_{file_name}.pkl', 'ab') as file:
                pickle.dump(result, file)
                print(">>>>>> Pickle appended")

            full_result.append(result)
            page_number += 1

        parsed_document = full_result
    return parsed_document   # langchain Document type
### [End] Main Fucntions with pdfminer.six ###########################################################################################

def add_firstline_in_splitted_text(origin_splitted_text:str):
    lv1 = origin_splitted_text.metadata["First Division"]
    lv2 = origin_splitted_text.metadata["Second Division"]
    title = origin_splitted_text.metadata["File Name"]
    origin_page_content = origin_splitted_text.page_content
    first_sentence = f"This page explains {title}, that belongs to catogories of {lv1} and {lv2}."
    new_page_content = f'''{first_sentence}/n{origin_page_content}'''
    origin_matadata = origin_splitted_text.metadata
    return Document(page_content=new_page_content, metadata=origin_matadata)

In [ ]:
import chromadb
import pandas as pd
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

def read_vectordb_as_df(db_path:str):
    result = []
    client = chromadb.PersistentClient(path=db_path)
    for collection in client.list_collections():
        data = collection.get(include=['embeddings','documents', 'metadatas'])
        result.append(data)
        df = pd.DataFrame({"ids":data["ids"],
                        #    "embeddings":data["embeddings"],
                           "metadatas":data["metadatas"],
                           "documents":data["documents"]})
        df["first_div"] = df["metadatas"].apply(lambda x: x["First Division"])
        df["second_div"] = df["metadatas"].apply(lambda x: x["Second Division"])
        df["filename"] = df["metadatas"].apply(lambda x: x["File Name"])
        df = df[["ids", "first_div", "second_div","filename","documents", "metadatas"]]
    return df


def delete_document(filename:str, db_path:str):
  vector_store = Chroma(collection_name="collection_01", persist_directory=db_path, embedding_function=OllamaEmbeddings(model="bge-m3:latest"))
  del_ids = vector_store.get(where={'File Name':filename})["ids"]
  vector_store.delete(del_ids)
  print("Document is deleted")


In [ ]:
def split_into_ten_sublists(lst):
    # Determine the size of each sublist
    avg = len(lst) // 10
    remainder = len(lst) % 10
    sublists = []
    start = 0

    for i in range(10):
        end = start + avg + (1 if i < remainder else 0)
        sublists.append(lst[start:end])
        start = end

    sublists = [i for i in sublists if i != []]

    return sublists

lst1 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
lst2 = [1, 2, 3, 4, 5, 6, 7]
result1 = split_into_ten_sublists(lst1)
print(result1)
result2 = split_into_ten_sublists(lst2)
print(result2)

[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11], [12], [13], [14], [15]]
[[1], [2], [3], [4], [5], [6], [7]]


In [ ]:
lv1_dir = "/content/drive/MyDrive/Rules"     # 최상단 엄마 폴더
total_paths = []
total_paths = main_filepath_extractor(path=lv1_dir)  # 모든 PDF의 Full Path를 리스트에 담기
total_paths = list(set(total_paths))
total_paths.sort()
print(total_paths)
print(len(total_paths))
res = extract_level_name(path=total_paths[-1])
print(res)

['/content/drive/MyDrive/Rules/ABS/00-part-1A-ships-jul24_Conditions of Classification.pdf', '/content/drive/MyDrive/Rules/ABS/00-part-1C-lhsc-jul24.pdf', '/content/drive/MyDrive/Rules/ABS/00-part-1D-alternative-jan24_ Alternative Arrangements, Novel Concepts and New Technologies.pdf', '/content/drive/MyDrive/Rules/ABS/00-part-2-jul24_Materials and Welding.pdf', '/content/drive/MyDrive/Rules/BV/Ammonia-fuelled ships - Tentative Rules_671-NR_2024-09.pdf', '/content/drive/MyDrive/Rules/BV/Approval of computerized maintenance management systems used on board ships_496-NR_2022-07.pdf', '/content/drive/MyDrive/Rules/BV/Approval of service suppliers_533-NR_2024-02.pdf', '/content/drive/MyDrive/Rules/BV/Approval of the manufacturing process of metallic materials_480-NR_2024-07.pdf', '/content/drive/MyDrive/Rules/BV/Approval testing of welders_476-NR_2022-01.pdf', '/content/drive/MyDrive/Rules/BV/Classification and certification of high integrity protection system (HIPS)_524-NI_2006-11.pdf', '

# 파싱

In [ ]:
for path in tqdm(total_paths):
  print(">>> 중복 파일 체크")
  target_filename = extract_level_name(path=path)[-1]

  print(target_filename)
  picklefiles = os.listdir('/content/drive/MyDrive/Parsed_text')
  picklefiles = [i.replace(".pkl", "") for i in picklefiles]
  picklefiles = [i.replace("parsed_text_", "") for i in picklefiles]


  if target_filename not in picklefiles:

    print("============= MAIN PARSER ============")
    parsed_text = main_parser(path=path, lang="en")
    with open(f'/content/drive/MyDrive/Parsed_text/parsed_text_{target_filename}.pkl', 'wb') as file:
      pickle.dump(parsed_text, file)
      print(f"Parsing is Completed _ {target_filename}")

  else:
    print("기 파싱된 문서입니다.")

print("All Parsings are Completed")


  0%|          | 0/239 [00:00<?, ?it/s]

>>> 중복 파일 체크
00-part-1A-ships-jul24_Conditions of Classification
기 파싱된 문서입니다.
>>> 중복 파일 체크
00-part-1C-lhsc-jul24
기 파싱된 문서입니다.
>>> 중복 파일 체크
00-part-1D-alternative-jan24_ Alternative Arrangements, Novel Concepts and New Technologies
기 파싱된 문서입니다.
>>> 중복 파일 체크
00-part-2-jul24_Materials and Welding
기 파싱된 문서입니다.
>>> 중복 파일 체크
Ammonia-fuelled ships - Tentative Rules_671-NR_2024-09
기 파싱된 문서입니다.
>>> 중복 파일 체크
Approval of computerized maintenance management systems used on board ships_496-NR_2022-07
기 파싱된 문서입니다.
>>> 중복 파일 체크
Approval of service suppliers_533-NR_2024-02
기 파싱된 문서입니다.
>>> 중복 파일 체크
Approval of the manufacturing process of metallic materials_480-NR_2024-07
기 파싱된 문서입니다.
>>> 중복 파일 체크
Approval testing of welders_476-NR_2022-01
기 파싱된 문서입니다.
>>> 중복 파일 체크
Classification and certification of high integrity protection system (HIPS)_524-NI_2006-11
기 파싱된 문서입니다.
>>> 중복 파일 체크
Classification of compressed natural gas carriers_517-NR_2007-04
기 파싱된 문서입니다.
>>> 중복 파일 체크
Classification of floating gas u


  0%|          | 0.00/4.00M [00:00<?, ?iB/s]
  0%|          | 3.07k/4.00M [00:00<04:42, 14.2kiB/s]
  1%|          | 35.8k/4.00M [00:00<00:48, 82.2kiB/s]
  2%|▏         | 68.6k/4.00M [00:00<00:38, 103kiB/s] 
  2%|▏         | 85.0k/4.00M [00:00<00:36, 107kiB/s]
  3%|▎         | 101k/4.00M [00:01<00:34, 112kiB/s] 
  3%|▎         | 118k/4.00M [00:01<00:41, 93.5kiB/s]
  4%|▍         | 151k/4.00M [00:01<00:41, 91.9kiB/s]
  4%|▍         | 167k/4.00M [00:01<00:43, 87.2kiB/s]
  5%|▍         | 183k/4.00M [00:02<00:46, 82.4kiB/s]
  5%|▍         | 200k/4.00M [00:02<00:44, 86.3kiB/s]
  5%|▌         | 216k/4.00M [00:02<00:45, 82.9kiB/s]
  6%|▌         | 232k/4.00M [00:02<00:46, 80.7kiB/s]
  6%|▌         | 249k/4.00M [00:02<00:48, 78.2kiB/s]
  7%|▋         | 265k/4.00M [00:03<00:47, 78.8kiB/s]
  7%|▋         | 282k/4.00M [00:03<00:44, 83.8kiB/s]
  7%|▋         | 298k/4.00M [00:03<00:45, 81.1kiB/s]
  8%|▊         | 314k/4.00M [00:03<00:46, 79.4kiB/s]
  8%|▊         | 331k/4.00M [00:03<00:47, 77.2kiB/

download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar



  0%|          | 0.00/10.2M [00:00<?, ?iB/s]
  0%|          | 3.07k/10.2M [00:00<13:12, 12.9kiB/s]
  0%|          | 35.8k/10.2M [00:00<02:12, 76.8kiB/s]
  1%|          | 68.6k/10.2M [00:00<01:44, 97.1kiB/s]
  1%|          | 85.0k/10.2M [00:00<01:39, 102kiB/s] 
  1%|          | 101k/10.2M [00:01<01:34, 107kiB/s] 
  1%|          | 118k/10.2M [00:01<01:52, 90.1kiB/s]
  1%|▏         | 151k/10.2M [00:01<01:53, 88.6kiB/s]
  2%|▏         | 167k/10.2M [00:01<01:59, 84.1kiB/s]
  2%|▏         | 183k/10.2M [00:02<02:06, 79.4kiB/s]
  2%|▏         | 200k/10.2M [00:02<01:57, 85.3kiB/s]
  2%|▏         | 216k/10.2M [00:02<02:03, 81.4kiB/s]
  2%|▏         | 232k/10.2M [00:02<02:07, 78.7kiB/s]
  2%|▏         | 249k/10.2M [00:03<02:12, 75.4kiB/s]
  3%|▎         | 265k/10.2M [00:03<02:13, 74.7kiB/s]
  3%|▎         | 282k/10.2M [00:03<02:02, 81.4kiB/s]
  3%|▎         | 298k/10.2M [00:03<02:05, 79.1kiB/s]
  3%|▎         | 314k/10.2M [00:03<02:11, 75.6kiB/s]
  3%|▎         | 331k/10.2M [00:04<02:12, 74.7kiB

...

>>>>>> Pickle appended
>>>>>> Pickle appended
Parsing is Completed _ Resolution_MEPC.329_76_
>>> 중복 파일 체크
370_CONDITIONS OF SERVICE FOR CLASSIFICATION OF SHIPS AND REGISTRATION OF INSTALLATIONS_202406
기 파싱된 문서입니다.
>>> 중복 파일 체크
370_REGULATIONS FOR THE CLASSIFICATION AND REGISTRY OF SHIPS_202406
기 파싱된 문서입니다.
>>> 중복 파일 체크
370_REGULATIONS FOR THE ISSUE OF STATUTORY CERTIFICATES_202406
기 파싱된 문서입니다.
>>> 중복 파일 체크
370_RULES FOR THE AUDIT AND REGISTRATION OF SAFETY MANAGEMENT SYSTEMS_202406
기 파싱된 문서입니다.
>>> 중복 파일 체크
370_RULES FOR THE AUDIT AND REGISTRATION OF SHIP SECURITY MANAGEMENT SYSTEMS_202406
기 파싱된 문서입니다.
>>> 중복 파일 체크
370_RULES FOR THE INSPECTION AND REGISTRATION OF MARITIME LABOUR SYSTEMS_202406
기 파싱된 문서입니다.
>>> 중복 파일 체크
371_RULES FOR APPROVAL OF MANUFACTURERS AND SERVICE SUPPLIERS_202406
기 파싱된 문서입니다.
>>> 중복 파일 체크
380_RULES FOR THE SURVEY AND CONSTRUCTION OF STEEL SHIPS_part_a_202406
기 파싱된 문서입니다.
>>> 중복 파일 체크
380_RULES FOR THE SURVEY AND CONSTRUCTION OF STEEL SHIPS_part_b_202406
기 파싱된 문서

  0%|          | 0/25 [00:00<?, ?it/s]

>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
Parsing is Completed _ Focus on IMO - SOLAS, the International Convention for the Safety, of Life at Sea, 1974 (October 1998)
>>> 중복 파일 체크
SOLAS_volume-1184-I-18961-English
============= MAIN PARSER ============
폴더가 생성되었습니다: /content/drive/MyDrive/images/SOLAS_volume-1184-I-18961-English
[2024/11/12 03:39:08] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=T

  0%|          | 0/178 [00:00<?, ?it/s]

>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pickle appended
>>>>>> Pick

In [ ]:
filename = "service-experience"
with open(f'/content/drive/MyDrive/Parsed_text/parsed_text_{filename}.pkl', 'rb') as file:
    parsed_text = pickle.load(file)

parsed_text[5]

Document(metadata={'Page': 5, 'First Division': 'MANUAL', 'Second Division': 'MAN', 'File Name': 'service-experience', 'File Path': '/content/drive/MyDrive/MANUAL/MAN/service-experience.pdf'}, page_content='MAN Energy Solutions Service experience6 close (WVFC) has been introduced, and a gas channel relief valve (GCRV) has been added to the ME-GI Mk. 2 design. All ME-GI Mk. 2 changes have been introduced to simplify the design. Hereby both reliability and cost have been addressed. Pilot injection valves (PIV), with control valve, sludge valve, sensor, and high-pressure (HP) pipes, are standard on ME-GI Mk. 2 engines. We have added a sealing oil block valve (SOB) and a sealing oil sensor to the ME-GI Mk. 2 to accommodate single-cylinder gas cut-out. Fig. 1  illustrates the ME-GI Mk. 2 gas block design. Pilot injection valve The introduction of the PIV has been successful, see Fig. 2 . Not only has the specific pilot oil consumption (SPOC) Dual-fuel service experience ME-GI service experi

# 벡터DB 저장

In [ ]:
db_path = "/content/drive/MyDrive/db/chroma_db_02"
vector_store = Chroma(collection_name="collection_01", persist_directory=db_path, embedding_function=OllamaEmbeddings(model="bge-m3:latest"))
df = read_vectordb_as_df(db_path=db_path)
vectordb_filenames = df["filename"].unique().tolist()

picklefiles = os.listdir('/content/drive/MyDrive/Parsed_text')

for file in tqdm(picklefiles):
  filename = file.replace(".pkl", "")
  filename = filename.replace("parsed_text_", "")
  print(filename)

  if filename not in vectordb_filenames:
    print("go")
    try:
      with open(f'/content/drive/MyDrive/Parsed_text/parsed_text_{filename}.pkl', 'rb') as file:
          parsed_text = pickle.load(file)

      text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
      splitted_texts = text_splitter.split_documents(parsed_text)
      new_splitted_texts = [add_firstline_in_splitted_text(text) for text in splitted_texts]

      new_splitted_texts = split_into_ten_sublists(new_splitted_texts)
      vector_store = Chroma(collection_name="collection_01",embedding_function=OllamaEmbeddings(model="bge-m3:latest"), persist_directory=db_path)

      for sublist in tqdm(new_splitted_texts):
        print(sublist)
        vector_store.add_documents(documents = sublist)
        time.sleep(0.3)
    except: pass


  else:
    print("기 작업된 파일")

print(">>>>>>>>>>> All Savings are completed")


  0%|          | 0/261 [00:00<?, ?it/s]

WinGD-Portfolio-Engines_2024_Digital
기 작업된 파일
16-06-pres-kit_x92
기 작업된 파일
cimac2016_173_brueckl_paper_virtualdesign-and-simulation-in2strokemarineenginedevelopment
기 작업된 파일
wingd-paper_engine_selection_for_very_large_container_vessels_201609
기 작업된 파일
CRYOGENIC-TRANSFER-PUMPS-FOR-LIQUEFIED-GASES-web
기 작업된 파일
Catalogue_LNG T&T_WEB
기 작업된 파일
Catalogue_POMPES-PISTON-v2023_WEB
기 작업된 파일
HYDROCARBON-TURBO_EXPANDERS
기 작업된 파일
equipment-solutions-for-lng-and-lbl
기 작업된 파일
PART 14_2024_Structural Rules for Container Ships
기 작업된 파일
PART 15_2024_Structural Rules for Membrane Type Liquefied Natural Gas Carriers
기 작업된 파일
PART 7(CH 5, CH 6)_2024_Ships of Special Service(CH5, 6)
기 작업된 파일
PART 7_2024_Ships of Special Service(Ch1_4, 7_10)
기 작업된 파일
PART 12_2014_Common Structural Rules for Double Hull Oil Tankers
기 작업된 파일
Rules on cyber security for the classification of marine units_659-NR_2024-07
기 작업된 파일
Rules on materials and welding for the classification of marine units_216-NR_2024-07
기 작업된 파일
Strength

  0%|          | 0/10 [00:00<?, ?it/s]

[Document(metadata={'Page': 0, 'First Division': 'Rules', 'Second Division': 'IGC', 'File Name': 'Amendments to the International Code for the Construction and Equipment of Ships Carrying Liquefied Gases in Bulk(IGC CODE)_2014', 'File Path': '/content/drive/MyDrive/Rules/IGC/Amendments to the International Code for the Construction and Equipment of Ships Carrying Liquefied Gases in Bulk(IGC CODE)_2014.pdf'}, page_content='This page explains Amendments to the International Code for the Construction and Equipment of Ships Carrying Liquefied Gases in Bulk(IGC CODE)_2014, that belongs to catogories of Rules and IGC./nRESOLUTION MSC.370(93) (adopted on 22 May 2014) AMENDMENTS TO THE INTERNATIONAL CODE FOR THE CONSTRUCTION AND EQUIPMENT OF SHIPS CARRYING LIQUEFIED GASES IN BULK (IGC CODE)'), Document(metadata={'Page': 1, 'First Division': 'Rules', 'Second Division': 'IGC', 'File Name': 'Amendments to the International Code for the Construction and Equipment of Ships Carrying Liquefied Gases 

  0%|          | 0/10 [00:00<?, ?it/s]

[Document(metadata={'Page': 0, 'First Division': 'Rules', 'Second Division': 'IGC', 'File Name': 'International Code for the Construction and Equipment of Ships Carrying Liguefied Gases in Bulk(IGC Code)', 'File Path': '/content/drive/MyDrive/Rules/IGC/International Code for the Construction and Equipment of Ships Carrying Liguefied Gases in Bulk(IGC Code).pdf'}, page_content='This page explains International Code for the Construction and Equipment of Ships Carrying Liguefied Gases in Bulk(IGC Code), that belongs to catogories of Rules and IGC./nSHIPPING Treaty Series No. 15 (1998) 1983 Amendments to the International Convention for the Safety of Life at Sea, 1974 (MSC. 5(48)) Volume 3 International Code for the Construction and Equipment of Ships Carrying Liquefied Gases in Bulk (IGC Code) Adopted at London, 17 June 1983 [The Amendments entered into force on 1 July 1986] Presented to Parliament by the Secretary of State for Foreign and Commonwealth Affairs by Command of Her Majesty Ap

  0%|          | 0/10 [00:00<?, ?it/s]

[Document(metadata={'Page': 0, 'First Division': 'Rules', 'Second Division': 'IGF', 'File Name': 'Draft International Code of Safety for Ships Using Gases or Other Low_Flashpoint Fuels(IGF Code) _2013_rev 12.07.2013', 'File Path': '/content/drive/MyDrive/Rules/IGF/Draft International Code of Safety for Ships Using Gases or Other Low_Flashpoint Fuels(IGF Code) _2013_rev 12.07.2013.pdf'}, page_content='This page explains Draft International Code of Safety for Ships Using Gases or Other Low_Flashpoint Fuels(IGF Code) _2013_rev 12.07.2013, that belongs to catogories of Rules and IGF./nDRAFT INTERNATIONAL CODE OF SAFETY FOR SHIPS USING GASES  OR OTHER LOW-FLASHPOINT FUELS (IGF CODE)  (including round 1 summary)  INDEX  1\xa0 Preamble \xa0............................................................................................................................... ..\xa08\xa0 PART\xa0A\xa09\xa0 2\xa0 GENERAL ....................................................................................

# VectorDB 조회

In [ ]:
vector_store = Chroma(collection_name="collection_01", persist_directory=db_path, embedding_function=OllamaEmbeddings(model="bge-m3:latest"))
df = read_vectordb_as_df(db_path=db_path)
print(vector_store)
print(df.shape)
print(df["first_div"].unique())
print(df["second_div"].unique())
print(df["filename"].unique()[-3:])
print(len(df["filename"].unique()))

(113021, 6)
['MANUAL' 'Rules' 'PORT']
['Common' 'Integrated Smart Ship(ISS)' 'ABS' 'DNV' 'KR' 'MARPOL' 'SOLAS'
 'BV' 'Port Regulation' 'Win GD' 'Cryostar' 'LR' 'NK' 'MAN' 'IGC' 'IGF']
['Amendments to the International Code for the Construction and Equipment of Ships Carrying Liquefied Gases in Bulk(IGC CODE)_2014'
 'International Code for the Construction and Equipment of Ships Carrying Liguefied Gases in Bulk(IGC Code)'
 'Draft International Code of Safety for Ships Using Gases or Other Low_Flashpoint Fuels(IGF Code) _2013_rev 12.07.2013']
262


In [ ]:
df.tail()

NameError: name 'df' is not defined

In [ ]:
query = """
what is the optimized specification of 10G95ME-C10.5-GI?
"""
res = vector_store.similarity_search_with_relevance_scores(query=query, k=5)
res

[(Document(metadata={'File Name': '00-part-2-jul24_Materials and Welding', 'File Path': '/content/drive/MyDrive/Rules/ABS/00-part-2-jul24_Materials and Welding.pdf', 'First Division': 'Rules', 'Page': 48, 'Second Division': 'ABS'}, page_content='This page explains 00-part-2-jul24_Materials and Welding, that belongs to catogories of Rules and ABS./n|Specimen Size|10 x 7.5 mm (0.394 x 0.295 in.)|10 x 5.0 mm (0.394 x 0.197 in.)|10 x 2.5 (0.394 x 0.098 in.)|\n|-------------|-------------------------------|-------------------------------|----------------------------|\n|Required Energy|5E/6|2E/3|E/2|'),
  0.23184350874542248),
 (Document(metadata={'File Name': 'Rules on materials and welding for the classification of marine units_216-NR_2024-07', 'File Path': '/content/drive/MyDrive/Rules/BV/Rules on materials and welding for the classification of marine units_216-NR_2024-07.pdf', 'First Division': 'Rules', 'Page': 59, 'Second Division': 'BV'}, page_content='This page explains Rules on mater

# 벡터DB에서 개별 문서 삭제

In [ ]:
# print(df.shape)
# try:
#   filename = 'Amendments to the International Code for the Construction and Equipment of Ships Carrying Liquefied Gases in Bulk(IGC CODE)_2014'
#   delete_document(filename=filename, db_path=db_path)
#   vector_store = Chroma(collection_name="collection_01", persist_directory=db_path, embedding_function=OllamaEmbeddings(model="bge-m3:latest"))
#   df = read_vectordb_as_df(db_path=db_path)
#   print(df.shape)
# except:
#   print("문서가 없습니다.")
#   vector_store = Chroma(collection_name="collection_01", persist_directory=db_path, embedding_function=OllamaEmbeddings(model="bge-m3:latest"))
#   df = read_vectordb_as_df(db_path=db_path)
#   print(df.shape)

# DB 등 파일 삭제

In [ ]:
# !rm -rf /content/drive/MyDrive/db/chroma_db_01

In [ ]:
# ! rm -rf /content/drive/MyDrive/Rules/ABS
# ! rm -rf /content/drive/MyDrive/Rules/DNV
# ! rm -rf /content/drive/MyDrive/Rules/KR
# ! rm -rf /content/drive/MyDrive/Rules/BV

# ! rm -rf /content/drive/MyDrive/Rules/MARPOL
# ! rm -rf /content/drive/MyDrive/Rules/SOLAS

! rm -rf /content/drive/MyDrive/MANUAL/Cryostar

In [ ]:
# lst = [1, 2, 3, 4, 5, 6, 7]
# squares_generator = (i for i in lst)
# squares_generator


<generator object <genexpr> at 0x7e22c34a95b0>

In [ ]:
# for i in tqdm(squares_generator):
#   print(i)

0it [00:00, ?it/s]

1
2
3
4
5
6
7
